In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:

! kaggle datasets download aiswaryaramachandran/hindienglish-corpora

  0% 0.00/13.9M [00:00<?, ?B/s]
100% 13.9M/13.9M [00:00<00:00, 166MB/s]


In [ ]:
!unzip /content/hindienglish-corpora.zip

Archive:  /content/hindienglish-corpora.zip
  inflating: Hindi_English_Truncated_Corpus.csv  


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
data = pd.read_csv('/content/Hindi_English_Truncated_Corpus.csv')
data.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [ ]:
data.shape

(127607, 3)

In [ ]:
data.isnull().sum()

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

In [ ]:
data[data['english_sentence'].isnull()]

,source,english_sentence,hindi_sentence
37554,indic2012,NaN,सन् 330 ईसापूर्व में मकदूनिया (यूनान) के विजेत...
59804,indic2012,NaN,लेकिन उस समय इस्लाम का उदय नहीं हुआ था; ईरान क...


In [ ]:
data.dropna(inplace = True)

In [ ]:
data['source'].unique()

array(['ted', 'indic2012', 'tides'], dtype=object)

In [ ]:
data.duplicated().sum()

2778

In [ ]:
data.drop_duplicates(inplace  =True , keep = 'first')

In [ ]:
data.duplicated().sum()

0

In [ ]:
data['english_sentence'] = data['english_sentence'].apply(lambda x : x.lower())
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : x.lower())

In [ ]:
import re
data['english_sentence'] = data['english_sentence'].apply(lambda x : re.sub("'" , '' , x))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : re.sub("'" , '' , x))

In [ ]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
unwanted = string.punctuation
data['english_sentence'] = data['english_sentence'].apply(lambda x : ''.join(ch for ch in x if ch not in unwanted))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : ''.join(ch for ch in x if ch not in unwanted))

In [ ]:
from string import digits
remove_digits = str.maketrans('', '', digits)
data['english_sentence'] = data['english_sentence'].apply(lambda x : x.translate(remove_digits))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : x.translate(remove_digits))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [ ]:
data['english_sentence'] = data['english_sentence'].apply(lambda x : x.strip())
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : x.strip())
data['english_sentence'] = data['english_sentence'].apply(lambda x : re.sub(' +' , ' ' , x))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : re.sub(' +' , ' ' , x))

In [ ]:
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : 'START_ ' + x + ' END_')

In [ ]:
data.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...
1,ted,id like to tell you about one such child,START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...
2,indic2012,this percentage is even greater than the perce...,START_ यह प्रतिशत भारत में हिन्दुओं प्रतिशत से...
3,ted,what we really mean is that theyre bad at not ...,START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...
4,indic2012,the ending portion of these vedas is called up...,START_ इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता...


In [ ]:
all_eng_word = []
for sen in data['english_sentence']:
    for word in sen.split():
        if(word not in all_eng_word):
            all_eng_word.append(word)
print(len(all_eng_word))

72686


In [ ]:
all_hin_word = []
for sen in data['hindi_sentence']:
  for word in sen.split():
    if(word not in all_hin_word):
      all_hin_word.append(word)
print(len(all_hin_word))

81977


In [ ]:
data['len_english_sentence'] = data['english_sentence'].apply(lambda x : len(x.split()))
data['len_hindi_sentence'] = data['hindi_sentence'].apply(lambda x : len(x.split()))

In [ ]:
print(max(data['len_english_sentence']))
print(max(data['len_hindi_sentence']))

398
419


In [ ]:
data = data[data['len_english_sentence'] < 25]
print(data.shape)
data = data[data['len_hindi_sentence'] < 25]
print(data.shape)

(104072, 5)
(93429, 5)


In [ ]:
max_length_src=max(data['len_english_sentence'])
max_length_tar=max(data['len_hindi_sentence'])

In [ ]:
input_word = sorted(list(all_eng_word))
target_word = sorted(list(all_hin_word))
num_encoder_token = len(all_eng_word)
num_decoder_token = len(target_word)
num_encoder_token , num_decoder_token

(72686, 81977)

In [ ]:
num_decoder_token += 1
num_decoder_token

81978

In [ ]:
input_token_index  = dict([(word , i+1) for i,word in enumerate(input_word)])
target_token_index = dict([(word , i+1) for i,word in enumerate(target_word)])

In [ ]:
# input_token_index

In [ ]:
reverse_input_token_index = dict((i , word) for i,word in input_token_index.items())
reverse_target_token_index = dict((i , word) for i ,word in target_token_index.items())

In [ ]:
# reverse_input_token_index


In [ ]:
from sklearn.model_selection import train_test_split
X , y = data['english_sentence'] , data['hindi_sentence']
X_train , X_test ,y_train ,  y_test = train_test_split(X , y , test_size =0.3)
X_train.shape , X_test.shape

((65400,), (28029,))

In [ ]:
def generate_batch(x , y , batch_size = 128):
  while True:
    for j in range(len(x)):
      encoder_input_data = np.zeros((batch_size , max_length_src) , dtype= 'float32')
      decoder_input_data = np.zeros((batch_size , max_length_tar) , dtype= 'float32')
      decoder_target_data = np.zeros((batch_size , max_length_tar , num_decoder_token) ,dtype = 'float32')
      for i ,(input_text , target_text) in enumerate(zip(x[j:j+batch_size] , y[j:j+batch_size])):
        for t,word in enumerate(input_text.split()):
          encoder_input_data[i,t] = input_token_index[word]
        for t,word in enumerate(target_text.split()):
          if( t < len(target_text.split()) -1):
            decoder_input_data[i,t] = target_token_index[word]
          if(t > 0):
            decoder_target_data[i , t-1 , target_token_index[word]] = 1
      yield ([encoder_input_data , decoder_input_data] , decoder_target_data)

In [ ]:
latent_dim  = 300

In [ ]:
from keras.layers import LSTM , Input , Embedding ,Dense
encoder_inputs = Input(shape = (None , ))
enc_em = Embedding(num_encoder_token , latent_dim , mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim  , return_state = True)
encoder_outputs , state_h , state_c =  encoder_lstm(enc_em)

encoder_states = [state_h , state_c]

In [ ]:
from keras.models import Model

decoder_inputs = Input(shape = (None , ))
dec_em = Embedding(num_decoder_token , latent_dim, mask_zero = True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim , return_sequences=True, return_state= True)
decoder_outputs , _,_ =decoder_lstm(dec_em , initial_state= encoder_states)
decoder_outputs = Dense(num_decoder_token , activation= 'softmax')(decoder_outputs)
model= Model([encoder_inputs , decoder_inputs] , decoder_outputs)

In [ ]:
model.compile(optimizer='adam' , loss = 'categorical_crossentropy')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    21805800    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    24593400    ['input_2[0][0]']                
                                                                                              

In [ ]:
train_samples = len(X_train)
val_sample = len(X_test)
batch_size = 128
epochs = 25


In [ ]:
model.fit_generator(generator = generate_batch(X_train , y_train ,batch_size = batch_size),
                        steps_per_epoch = train_samples//batch_size ,
                          epochs = epochs   ,
                          validation_data = generate_batch(X_test , y_test , batch_size = batch_size),
                          validation_steps = val_sample//batch_size , )


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/25
510/510 [==============================] - 471s 901ms/step - loss: 2.4907 - val_loss: 4.8078
Epoch 2/25
510/510 [==============================] - 454s 891ms/step - loss: 0.8054 - val_loss: 4.3661
Epoch 3/25
510/510 [==============================] - 453s 889ms/step - loss: 0.4461 - val_loss: 3.9320
Epoch 4/25
510/510 [==============================] - 474s 930ms/step - loss: 0.3202 - val_loss: 4.2564
Epoch 5/25
510/510 [==============================] - 481s 943ms/step - loss: 0.2855 - val_loss: 4.1730
Epoch 6/25
510/510 [==============================] - 480s 941ms/step - loss: 0.2712 - val_loss: 4.0197
Epoch 7/25
510/510 [==============================] - 480s 941ms/step - loss: 0.2737 - val_loss: 4.0441
Epoch 8/25
510/510 [==============================] - 467s 917ms/step - loss: 0.2653 - val_loss: 4.1237
Epoch 9/25
510/510 [==============================] - 452s 887ms/step - loss: 0.2496 - val_loss: 4.1858
Epoch 10/25
168/510 [========>.....................] - ETA: 3:46

KeyboardInterrupt: ignored

In [ ]:
model.save_weights('nmt_weights.h5')